# Reinforcement Learning Basics Tutorial

This tutorial is adapted from [python tutorial](https://medium.com/@m.alzantot/deep-reinforcement-learning-demysitifed-episode-2-policy-iteration-value-iteration-and-q-978f9e89ddaa), which shows how to implement value iteration and tabular Q learning
on the FrozenLake task from the `OpenAI Gym <https://gym.openai.com/>`

**Task**

In `FrozenLake-v0`, the agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into the water. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile. 

![](https://cdn-images-1.medium.com/max/1600/1*MCjDzR-wfMMkS0rPqXSmKw.png)

As the agent observes the current state of the environment and chooses an action, the environment *transitions* to a new state, and also returns a reward that indicates the consequences of the action. In this task, the episode ends when you reach the goal or fall in a hole. You receive a reward of 1 if you reach the goal, and zero otherwise.

**Deterministic Environment**

Deterministic environment means that both state transition model and reward model are deterministic functions. If the agent while in a given state repeats a given action, it will always go the same next state and receive the same reward value.

**Stochastic Environment**

In a stochastic environment there is uncertainty about the actions effect. When the agent repeats doing the same action in a given state, the new state and received reward may not be the same each time. For example, a robot which tries to move forward but because of the imperfection in the robot operation or other factors in the environment (e.g. slippery floor), sometimes the action forward will make it move forward but in sometimes, it will move to left or right.

The environment `FrozenLake-v0` from OpenAI Gym is a stochastic environment. FrozenLake-v0 is considered "solved" when the agent obtains an average reward of at least 0.78 over 100 consecutive episodes. We can also make a deterministic version of the environment.

In [1]:
from gym.envs.registration import register
register(
    id='FrozenLakeNotSlippery-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery': False},
    max_episode_steps=100,
    reward_threshold=0.8196, # optimum = .8196, changing this seems have no influence
)

**Value Iteration**

Value iteration computes the optimal state value function by iteratively improving the estimate of V(s). The algorithm initialize V(s) to arbitrary random values. It repeatedly updates the Q(s, a) and V(s) values until they converges. Value iteration is guaranteed to converge to the optimal values. This algorithm is shown in the following pseudo-code:
<img src="https://cdn-images-1.medium.com/max/1600/1*S-a_T-k5hXYhinq9758xCQ.png" alt="value iteration algorithm" style="width: 600px;"/>


In [2]:
import numpy as np
import gym
from gym import wrappers


def evaluate_policy(env, policy, render = False):
    """ Evaluates policy by using it to run an episode and finding its
    total reward.
    args:
    env: gym environment.
    policy: the policy to be used.
    gamma: discount factor.
    render: boolean to turn rendering on/off.
    returns:
    total reward: real value of the total reward recieved by agent under policy.
    """
    obs = env.reset()
    total_reward = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += reward
        if done:
            break
    return total_reward


def value_iteration(env, gamma = 1.0):
    """ Value-iteration algorithm """
    v = np.zeros(env.observation_space.n)  # initialize value-function
    max_iterations = 100000
    eps = 1e-30
    for i in range(max_iterations):
        delta = 0
        ## TODO: iteratively update the value estimate
        ## hint: env.unwrapped.P[s][a] provides the transition probability 
        ## hint: env.action_space.n is the number of actions
        prev_v = np.copy(v)
        for s in range(env.observation_space.n):
            q_sa = [sum([p*(r + gamma*prev_v[s_]) for p, s_, r, _ in env.unwrapped.P[s][a]]) for a in range(env.action_space.n)] 
            v[s] = max(q_sa)
            delta = max(delta, np.fabs(prev_v[s] - v[s]))
        if (delta < eps):
            print ('Value-iteration converged at iteration# %d.' %(i+1))
            break
    return v

def extract_policy(v, gamma = 1.0):
    """ Extract the policy given a value-function """
    policy = np.zeros(env.observation_space.n)
    for s in range(env.observation_space.n):
        ## TODO: extra the optimal policy based on the optimal value estimate
        ## hint: env.unwrapped.P[s][a] provides the transition probability 
        ## hint: env.action_space.n is the number of actions
        q_sa = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            for next_sr in env.unwrapped.P[s][a]:
                # next_sr is a tuple of (probability, next state, reward, done)
                p, s_, r, _ = next_sr
                q_sa[a] += (p * (r + gamma * v[s_]))
        policy[s] = np.argmax(q_sa)
    return policy


In [4]:
env_name  = 'FrozenLakeNotSlippery-v0'
#env_name  = 'FrozenLake-v0'
gamma = 0.99
env = gym.make(env_name)
env.seed(598)
np.random.seed(598)
optimal_v = value_iteration(env, gamma);
optimal_policy = extract_policy(optimal_v, gamma)
optimal_policy_score = np.mean([evaluate_policy(env, optimal_policy) for _ in range(100)])
print('Policy average score ', optimal_policy_score)
print('One episode score', evaluate_policy(env, optimal_policy, render=True))
env.close()

/Users/guoyijie/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


Value-iteration converged at iteration# 996.
Policy average score  0.81

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HF

**Q Learning**

Q-Learning is an example of model-free learning algorithm. It does not assume that agent knows anything about the state-transition and reward models. However, the agent will discover what are the good and bad actions by trial and error.

![](http://incompleteideas.net/book/first/ebook/pseudotmp9.png)

**Exploration vs exploitation**

An important question is how does the agent select actions during learning. Should the agent trust the learnt values of Q(s, a) enough to select actions based on it ? or try other actions hoping this may give it a better reward. This is known as the exploration vs exploitation dilemma.

A simple approach is known as the 𝛆-greedy approach where at each step. With small probability 𝛜, the agent will pick a random action (explore) or with probability (1-𝛜) the agent will select an action according to the current estimate of Q-values. 𝛜 value can be decreased overtime as the agent becomes more confident with its estimate of Q-values.

In [5]:
def choose_action(q, eps, num_actions):
    ## TODO: implement epsilon-greedy for action selection
    ## hint: np.random.uniform(0,1) < eps could be the condition for random action
    if np.random.uniform(0, 1) < eps:
        action = np.random.choice(env.action_space.n)
    else:
        greedy_actions = np.where(q==np.max(q))[0]
        action = np.random.choice(greedy_actions)
    return action

def q_learning(env, alpha, gamma, eps, iter_max, t_max):
    q_table = np.zeros((env.observation_space.n, env.action_space.n))
    total_reward_list = []
    for i in range(iter_max):
        total_reward = 0
        s = env.reset() 
        for j in range(t_max):
            action = choose_action(q_table[s], eps, env.action_space.n)
            next_s, reward, done, _ = env.step(action)
            total_reward += reward
            ## TODO: update Q table
            q_table[s][action] = q_table[s][action] + alpha * (reward + gamma *  np.max(q_table[next_s]) - q_table[s][action])
            s=next_s
            if done:
                total_reward_list.append(total_reward)
                break
        if i % 100 == 0:
            print('Iteration #%d -- Total reward = %g' %(i+1, np.mean(total_reward_list[-100:])))
    return np.argmax(q_table, axis=1)
    

In [7]:
env_name = 'FrozenLakeNotSlippery-v0'
#env_name = 'FrozenLake-v0'
env = gym.make(env_name)
env.seed(598)
np.random.seed(598)


solution_policy = q_learning(env=env, alpha=0.5, gamma=0.96, eps=0.02, iter_max=10000, t_max=10000)
solution_policy_score = np.mean([evaluate_policy(env, solution_policy) for _ in range(100)])
print("Policy average score", solution_policy_score)
print('One episode score', evaluate_policy(env, solution_policy, True))
env.close()

Iteration #1 -- Total reward = 0
Iteration #101 -- Total reward = 0.01
Iteration #201 -- Total reward = 0.03
Iteration #301 -- Total reward = 0.02
Iteration #401 -- Total reward = 0.06
Iteration #501 -- Total reward = 0.4
Iteration #601 -- Total reward = 0.41
Iteration #701 -- Total reward = 0.39
Iteration #801 -- Total reward = 0.48
Iteration #901 -- Total reward = 0.38
Iteration #1001 -- Total reward = 0.48
Iteration #1101 -- Total reward = 0.35
Iteration #1201 -- Total reward = 0.49
Iteration #1301 -- Total reward = 0.48
Iteration #1401 -- Total reward = 0.44
Iteration #1501 -- Total reward = 0.33
Iteration #1601 -- Total reward = 0.57
Iteration #1701 -- Total reward = 0.44
Iteration #1801 -- Total reward = 0.55
Iteration #1901 -- Total reward = 0.47
Iteration #2001 -- Total reward = 0.4
Iteration #2101 -- Total reward = 0.38
Iteration #2201 -- Total reward = 0.5
Iteration #2301 -- Total reward = 0.44
Iteration #2401 -- Total reward = 0.3
Iteration #2501 -- Total reward = 0.45
Itera